# 科学技術計算 1 レポートテンプレート


In [ ]:
import numpy as np
from numpy.linalg import inv, norm, solve, det, matrix_rank, cond
from numpy import diag
rng = np.random.default_rng()

import scipy

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["savefig.bbox"] = "tight"

import warnings
warnings.simplefilter('ignore', FutureWarning)

from typing import Tuple, List, Optional, Union
from tqdm.auto import tqdm


## 課題01-1-1

## 課題01-1-2

## 課題01-1-3

## 課題01-2